# Import

In [ ]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 opencv-python mediapipe sklearn matplotlib

In [18]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# Setup Mediapipe

In [19]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [20]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [21]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face tesselations
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

# Test Mediapipe

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Extracting Mediapipe Keypoint Values

In [22]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Setup Folders for Data Collection

In [23]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [13]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# Collect Data & Keypoint Values for Training and Testing

In [14]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop through actions
    for action in actions:
        # Loop through videos
        for sequence in range(no_sequences):
            # Loop through video length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

# Create Labels and Features & Train-Test Split

In [24]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [25]:
label_map = {label:num for num, label in enumerate(actions)}

In [26]:
label_map

{'hello': 0, 'thanks': 1, 'iloveyou': 2}

In [27]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
np.array(sequences).shape

(90, 30, 1662)

In [29]:
np.array(labels).shape

(90,)

In [30]:
X = np.array(sequences)

In [31]:
X.shape

(90, 30, 1662)

In [32]:
y = to_categorical(labels).astype(int)

In [33]:
y

array([[1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0,

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [35]:
y_test.shape

(5, 3)

# Build and Train LSTM Neural Network

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [37]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [38]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [39]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [31]:
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])

Epoch 1/1000
3/3 [==============================] - 9s 404ms/step - loss: 1.8440 - categorical_accuracy: 0.3294
Epoch 2/1000
3/3 [==============================] - 1s 437ms/step - loss: 10.9380 - categorical_accuracy: 0.2824
Epoch 3/1000
3/3 [==============================] - 1s 395ms/step - loss: 14.5823 - categorical_accuracy: 0.3294
Epoch 4/1000
3/3 [==============================] - 1s 432ms/step - loss: 19.2245 - categorical_accuracy: 0.2824
Epoch 5/1000
3/3 [==============================] - 1s 407ms/step - loss: 33.0619 - categorical_accuracy: 0.3529
Epoch 6/1000
3/3 [==============================] - 1s 398ms/step - loss: 22.3128 - categorical_accuracy: 0.3647
Epoch 7/1000
3/3 [==============================] - 1s 386ms/step - loss: 17.1159 - categorical_accuracy: 0.2941
Epoch 8/1000
3/3 [==============================] - 1s 402ms/step - loss: 23.7103 - categorical_accuracy: 0.3882
Epoch 9/1000
3/3 [==============================] - 1s 429ms/step - loss: 32.2776 - categorical_a

3/3 [==============================] - 1s 362ms/step - loss: 6.1078 - categorical_accuracy: 0.3647
Epoch 144/1000
3/3 [==============================] - 1s 374ms/step - loss: 6.3107 - categorical_accuracy: 0.3529
Epoch 145/1000
3/3 [==============================] - 1s 458ms/step - loss: 5.5604 - categorical_accuracy: 0.3647
Epoch 146/1000
3/3 [==============================] - 1s 389ms/step - loss: 3.7031 - categorical_accuracy: 0.4824
Epoch 147/1000
3/3 [==============================] - 1s 367ms/step - loss: 4.0441 - categorical_accuracy: 0.3765
Epoch 148/1000
3/3 [==============================] - 1s 443ms/step - loss: 2.3127 - categorical_accuracy: 0.4353
Epoch 149/1000
3/3 [==============================] - 1s 400ms/step - loss: 1.8926 - categorical_accuracy: 0.5059
Epoch 150/1000
3/3 [==============================] - 1s 346ms/step - loss: 2.5687 - categorical_accuracy: 0.4706
Epoch 151/1000
3/3 [==============================] - 1s 353ms/step - loss: 2.4038 - categorical_accura

3/3 [==============================] - 1s 417ms/step - loss: 5.6979 - categorical_accuracy: 0.2588
Epoch 286/1000
3/3 [==============================] - 2s 635ms/step - loss: 4.5315 - categorical_accuracy: 0.3647
Epoch 287/1000
3/3 [==============================] - 2s 550ms/step - loss: 4.9779 - categorical_accuracy: 0.3882
Epoch 288/1000
3/3 [==============================] - 2s 551ms/step - loss: 5.6230 - categorical_accuracy: 0.3176
Epoch 289/1000
3/3 [==============================] - 1s 474ms/step - loss: 24.7751 - categorical_accuracy: 0.3059
Epoch 290/1000
3/3 [==============================] - 1s 426ms/step - loss: 41.8014 - categorical_accuracy: 0.3412
Epoch 291/1000
3/3 [==============================] - 1s 433ms/step - loss: 46.5510 - categorical_accuracy: 0.3176
Epoch 292/1000
3/3 [==============================] - 1s 464ms/step - loss: 47.2203 - categorical_accuracy: 0.2471
Epoch 293/1000
3/3 [==============================] - 1s 502ms/step - loss: 50.8148 - categorical_a

Epoch 428/1000
3/3 [==============================] - 1s 448ms/step - loss: 1.2432 - categorical_accuracy: 0.3059
Epoch 429/1000
3/3 [==============================] - 1s 384ms/step - loss: 1.4778 - categorical_accuracy: 0.4353
Epoch 430/1000
3/3 [==============================] - 1s 428ms/step - loss: 1.7620 - categorical_accuracy: 0.3176
Epoch 431/1000
3/3 [==============================] - 1s 410ms/step - loss: 1.6153 - categorical_accuracy: 0.3882
Epoch 432/1000
3/3 [==============================] - 1s 390ms/step - loss: 1.5447 - categorical_accuracy: 0.4000
Epoch 433/1000
3/3 [==============================] - 1s 414ms/step - loss: 1.2346 - categorical_accuracy: 0.3882
Epoch 434/1000
3/3 [==============================] - 1s 487ms/step - loss: 1.2122 - categorical_accuracy: 0.4588
Epoch 435/1000
3/3 [==============================] - 1s 449ms/step - loss: 1.4413 - categorical_accuracy: 0.3529
Epoch 436/1000
3/3 [==============================] - 1s 418ms/step - loss: 1.4653 - cat

Epoch 571/1000
3/3 [==============================] - 1s 394ms/step - loss: 1.0429 - categorical_accuracy: 0.4706
Epoch 572/1000
3/3 [==============================] - 1s 398ms/step - loss: 1.1015 - categorical_accuracy: 0.5294
Epoch 573/1000
3/3 [==============================] - 1s 433ms/step - loss: 1.0393 - categorical_accuracy: 0.5412
Epoch 574/1000
3/3 [==============================] - 1s 441ms/step - loss: 1.0579 - categorical_accuracy: 0.6000
Epoch 575/1000
3/3 [==============================] - 1s 416ms/step - loss: 1.3087 - categorical_accuracy: 0.5176
Epoch 576/1000
3/3 [==============================] - 1s 371ms/step - loss: 1.5523 - categorical_accuracy: 0.4706
Epoch 577/1000
3/3 [==============================] - 1s 377ms/step - loss: 1.1715 - categorical_accuracy: 0.4471
Epoch 578/1000
3/3 [==============================] - 1s 407ms/step - loss: 1.4713 - categorical_accuracy: 0.4588
Epoch 579/1000
3/3 [==============================] - 1s 363ms/step - loss: 1.4538 - cat

Epoch 714/1000
3/3 [==============================] - 1s 355ms/step - loss: 0.9246 - categorical_accuracy: 0.5647
Epoch 715/1000
3/3 [==============================] - 1s 345ms/step - loss: 1.0503 - categorical_accuracy: 0.5647
Epoch 716/1000
3/3 [==============================] - 1s 434ms/step - loss: 1.1545 - categorical_accuracy: 0.5294
Epoch 717/1000
3/3 [==============================] - 1s 438ms/step - loss: 1.1233 - categorical_accuracy: 0.4706
Epoch 718/1000
3/3 [==============================] - 1s 432ms/step - loss: 0.9741 - categorical_accuracy: 0.5529
Epoch 719/1000
3/3 [==============================] - 1s 505ms/step - loss: 0.9537 - categorical_accuracy: 0.5176
Epoch 720/1000
3/3 [==============================] - 1s 470ms/step - loss: 0.9101 - categorical_accuracy: 0.4706
Epoch 721/1000
3/3 [==============================] - 1s 491ms/step - loss: 0.8170 - categorical_accuracy: 0.6353
Epoch 722/1000
3/3 [==============================] - 1s 391ms/step - loss: 0.8671 - cat

Epoch 857/1000
3/3 [==============================] - 1s 464ms/step - loss: 1.0586 - categorical_accuracy: 0.5294
Epoch 858/1000
3/3 [==============================] - 1s 445ms/step - loss: 0.8718 - categorical_accuracy: 0.5412
Epoch 859/1000
3/3 [==============================] - 1s 370ms/step - loss: 0.7861 - categorical_accuracy: 0.6706
Epoch 860/1000
3/3 [==============================] - 1s 457ms/step - loss: 0.9107 - categorical_accuracy: 0.5882
Epoch 861/1000
3/3 [==============================] - 1s 424ms/step - loss: 0.8286 - categorical_accuracy: 0.6235
Epoch 862/1000
3/3 [==============================] - 1s 396ms/step - loss: 0.8453 - categorical_accuracy: 0.6000
Epoch 863/1000
3/3 [==============================] - 1s 393ms/step - loss: 1.0419 - categorical_accuracy: 0.5529
Epoch 864/1000
3/3 [==============================] - 1s 542ms/step - loss: 0.8977 - categorical_accuracy: 0.5765
Epoch 865/1000
3/3 [==============================] - 1s 355ms/step - loss: 0.7553 - cat

Epoch 1000/1000
3/3 [==============================] - 1s 482ms/step - loss: 0.4986 - categorical_accuracy: 0.8353


In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

# Test Making Predictions

In [41]:
res = model.predict(X_test)

In [47]:
np.save("ytest",y_test)

In [42]:
actions[np.argmax(res[4])]

'thanks'

In [43]:
actions[np.argmax(y_test[4])]

'iloveyou'

# Save Weights & Model

In [36]:
model.save('action.h5')

In [217]:
del model

In [17]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [40]:
model.load_weights('action.h5')

# Evaluation using Confusion Matrix and Accuracy

In [37]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [38]:
yhat = model.predict(X_test)

In [39]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [ ]:
multilabel_confusion_matrix(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# Test in Real Time

In [21]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
plt.figure(figsize=(18,18))
plt.imshow(prob_viz(res, actions, image, colors))

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-sequence_length:]
        
        if len(sequence) == sequence_length:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            
            
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()